### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [35]:
# Dependencies and Setup
import pandas as pd
import numpy as np
# File to Load (Remember to Change These)
file_to_load = "purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data_df = pd.read_csv(file_to_load)
purchase_data_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


## Player Count

* Display the total number of players


In [36]:
# remove duplicate players
players_list = purchase_data_df["SN"].unique()
# count total number of players
number_of_players = len(players_list)
f"Total number of players: {number_of_players}"

'Total number of players: 576'

## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [37]:
# remove duplicate items
items_list = purchase_data_df["Item ID"].unique()
# calculate total number of unique items
number_of_items = len(items_list)
# calculate average price from all transactions
average_price = round(purchase_data_df["Price"].mean(), 2)
# calculate total number of purchases
purchases = len(purchase_data_df["Purchase ID"])
# calculate the sum of all purchases
revenue = purchase_data_df["Price"].sum()

In [38]:
#create summary dataframe for results
summary_table_df = pd.DataFrame({"Total Number of Players": number_of_players, "Total Number of Unique Items": number_of_items,
                    "Average Purchase Price": average_price,
                    "Total Number of Purchases": purchases,
                    "Total Revenue": revenue}, index=[0])
summary_table_df

,Total Number of Players,Total Number of Unique Items,Average Purchase Price,Total Number of Purchases,Total Revenue
0,576,179,3.05,780,2379.77


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [39]:
# create df for player info only
player_data_df = purchase_data_df.loc[:, ["SN", "Age", "Gender"]]
# drop duplicated from player df
player_data_unique_df = player_data_df.drop_duplicates()
# gather gender counts
gender_count = player_data_unique_df["Gender"].value_counts()
# gather gender percentages and round values to 2 decimals
gender_percentage = (player_data_unique_df["Gender"].value_counts() / number_of_players)*100
gender_percentage = gender_percentage.round(2)
# create df that combines counts and percentages
gender_demographics_df = pd.DataFrame({"Total Player Count": gender_count, "Percentage of Total Players": gender_percentage})
gender_demographics_df["Percentage of Total Players"] = gender_demographics_df["Percentage of Total Players"].map("{:.2f}%".format)
gender_demographics_df

,Total Player Count,Percentage of Total Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [40]:
# create data frames of purchasing info by gender
price_data_df = purchase_data_df.loc[:, ["Gender", "Price"]]

# calculate purchase count by gender
purchase_data_groupby_count_df = pd.DataFrame(price_data_df.groupby(["Gender"]).count())

# calculate avg. purchase price by gender
purchase_data_groupby_mean_df = pd.DataFrame(round(price_data_df.groupby(["Gender"]).mean(), 2))

# calculate total purchase value by gender
purchase_data_groupby_sum_df = pd.DataFrame(price_data_df.groupby(["Gender"]).sum())

# calculate avg. purchase total per person by gender
purchase_data_groupby_perplayer = round(purchase_data_groupby_sum_df["Price"] / gender_demographics_df["Total Player Count"], 2)
purchase_data_groupby_perplayer_df = pd.DataFrame({"Purchase Price Per Player": purchase_data_groupby_perplayer})

# create data frame containing results
purchasing_analysis_gender_df = pd.merge(purchase_data_groupby_count_df, purchase_data_groupby_sum_df, on="Gender")
purchasing_analysis_gender_df = purchasing_analysis_gender_df.rename(columns={"Price_x": "Purchase Count", "Price_y": "Total Purchase Value"})
purchasing_analysis_gender_df = pd.merge(purchasing_analysis_gender_df, purchase_data_groupby_mean_df, on="Gender")
purchasing_analysis_gender_df = purchasing_analysis_gender_df.rename(columns={"Price": "Average Purchase Price"})
purchasing_analysis_gender_df = pd.concat([purchasing_analysis_gender_df, purchase_data_groupby_perplayer_df], axis=1)
purchasing_analysis_gender_df["Total Purchase Value"] = purchasing_analysis_gender_df["Total Purchase Value"].map("${:.2f}".format)
purchasing_analysis_gender_df["Average Purchase Price"] = purchasing_analysis_gender_df["Average Purchase Price"].map("${:.2f}".format)
purchasing_analysis_gender_df["Purchase Price Per Player"] = purchasing_analysis_gender_df["Purchase Price Per Player"].map("${:.2f}".format)
purchasing_analysis_gender_df

,Purchase Count,Total Purchase Value,Average Purchase Price,Purchase Price Per Player
Female,113,$361.94,$3.20,$4.47
Male,652,$1967.64,$3.02,$4.07
Other / Non-Disclosed,15,$50.19,$3.35,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [41]:
#The below each broken into bins of 4 years (i.e. <10, 10-14, 15-19, etc.)
bins = [0, 9, 14, 19, 24, 29, 34, 39, 44, 49]
group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40-44", "45-49"]
purchase_data_df["Age Range"] = pd.cut(purchase_data_df["Age"], bins, labels=group_names, include_lowest=True)
age_data_df = purchase_data_df.loc[:, ["Age Range", "Price"]]

#Average Purchase Price
age_avg = round(age_data_df.groupby(["Age Range"]).mean(), 2)
age_avg_df = pd.DataFrame(age_avg)

#Total Purchase Value
age_sum_df = pd.DataFrame(age_data_df.groupby(["Age Range"]).sum())
age_sum_df

#Average Purchase Total per Person by Age Group
    # create df for player info only
age_range_data_df = purchase_data_df.loc[:, ["SN", "Age Range", "Gender"]]
    # drop duplicated from player df
age_data_unique_df = age_range_data_df.drop_duplicates()
    # gather gender counts
age_count = age_data_unique_df["Age Range"].value_counts()
age_price_perperson = round(age_sum_df["Price"] / age_count, 2)
age_price_perperson_df = pd.DataFrame({"Purchase Price Per Player": age_price_perperson})

#Summary data frame
combined = pd.merge(age_sum_df, age_avg_df, on="Age Range")
combined = combined.rename(columns={"Price_x": "Total Purchase Value", "Price_y": "Average Purchase Price"})
age_demographics_df = pd.concat([combined, age_price_perperson_df], axis=1)
age_demographics_df["Total Purchase Value"] = age_demographics_df["Total Purchase Value"].map("${:.2f}".format)
age_demographics_df["Average Purchase Price"] = age_demographics_df["Average Purchase Price"].map("${:.2f}".format)
age_demographics_df["Purchase Price Per Player"] = age_demographics_df["Purchase Price Per Player"].map("${:.2f}".format)
age_demographics_df

,Total Purchase Value,Average Purchase Price,Purchase Price Per Player
<10,$77.13,$3.35,$4.54
10-14,$82.78,$2.96,$3.76
15-19,$412.89,$3.04,$3.86
20-24,$1114.06,$3.05,$4.32
25-29,$293.00,$2.90,$3.81
30-34,$214.00,$2.93,$4.12
35-39,$147.67,$3.60,$4.76
40-44,$36.54,$3.04,$3.32
45-49,$1.70,$1.70,$1.70


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [42]:
#Identify the the top 5 spenders in the game by total purchase value, then list (in a table):
SN_Price_df = purchase_data_df.loc[:, ["SN", "Price"]]

#SN
SN_Price_grouped_df = SN_Price_df.groupby(["SN"])

#Purchase Count
SN_Price_count = SN_Price_grouped_df.count()["Price"]

#Average Purchase Price
SN_Price_mean = round(SN_Price_grouped_df.mean()["Price"], 2)
#Total Purchase Value
SN_Price_sum = SN_Price_grouped_df.sum()["Price"]
Top_Spenders_df = pd.DataFrame({"Purchase Count": SN_Price_count, "Total Purchase Value": SN_Price_sum, "Average Purchase Price": SN_Price_mean}).sort_values("Total Purchase Value", ascending=False)
Top_Spenders_df = Top_Spenders_df.head(5)
Top_Spenders_df["Total Purchase Value"] = Top_Spenders_df["Total Purchase Value"].map("${:.2f}".format)
Top_Spenders_df["Average Purchase Price"] = Top_Spenders_df["Average Purchase Price"].map("${:.2f}".format)
Top_Spenders_df

,Purchase Count,Total Purchase Value,Average Purchase Price
SN,,,
Lisosia93,5,$18.96,$3.79
Idastidru52,4,$15.45,$3.86
Chamjask73,3,$13.83,$4.61
Iral74,4,$13.62,$3.40
Iskadarya95,3,$13.10,$4.37


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, average item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [56]:
#Retrieve the Item ID, Item Name, and Item Price columns 
ItemID_ItemName_Price_df = purchase_data_df.loc[:, ["Item ID", "Item Name", "Price"]]
#Group by Item ID and Item Name.
Item_grouped = ItemID_ItemName_Price_df.groupby(["Item ID", "Item Name"])
#purchase count
Item_count = Item_grouped.count()["Price"]
#total purchase value
Item_sum = Item_grouped.sum()["Price"]
#average item price
Item_avg = round(Item_grouped.mean()["Price"], 2)
#summary data frame
Popular_Items_df = pd.DataFrame({"Purchase Count": Item_count, "Item Price": Item_avg, "Total Purchase Value": Item_sum}).sort_values("Purchase Count", ascending=False)
Popular_Items_df["Item Price"] = Popular_Items_df["Item Price"].map("${:.2f}".format)
Popular_Items_df["Total Purchase Value"] = Popular_Items_df["Total Purchase Value"].map("${:.2f}".format)
Most_Popular_Items_df = Popular_Items_df.head(5)
Most_Popular_Items_df

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [57]:
Profitable_Items_df = pd.DataFrame({"Purchase Count": Item_count, "Item Price": Item_avg, "Total Purchase Value": Item_sum}).sort_values("Total Purchase Value", ascending=False)
Profitable_Items_df["Item Price"] = Profitable_Items_df["Item Price"].map("${:.2f}".format)
Profitable_Items_df["Total Purchase Value"] = Profitable_Items_df["Total Purchase Value"].map("${:.2f}".format)
Most_Profitable_Items_df = Profitable_Items_df.head(5)
Most_Profitable_Items_df

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
